# HVplot HTML for folium

In [ ]:
import pandas as pd
import hvplot.pandas

In [ ]:
df = pd.read_parquet('timeseries/301_CA_SNTL.parquet')
df.head()

In [ ]:
df.info()

In [ ]:
df.to_json('timeseries/301_CA_SNTL.json')

In [ ]:
# Nice!
#hvplot.explorer(df)

In [ ]:
# NOTE: units info? 
# https://stackoverflow.com/questions/14688306/adding-meta-information-metadata-to-pandas-dataframe
plot = df.SNWD.hvplot()
plot

In [ ]:
# https://hvplot.holoviz.org/user_guide/Viewing.html?highlight=html#saving-plots
#type(plot)

In [ ]:
hvplot.save(plot, '301_CA_SNTL.html') #82 KB...embedding data? down to 65K with float32, try float16? different compression?
# But note that entire parquet file is memory usage: 102.8 KB...

In [ ]:
!ls -thrl *html

In [ ]:
!ls -thrl

In [ ]:
# OK, add the full HTML as a popup
# It seems you can use remote data with?
# https://altair-viz.github.io/
# https://python-visualization.github.io/folium/quickstart.html#Vincent/Vega-and-Altair/VegaLite-Markers ?
# https://altair-viz.github.io/user_guide/faq.html#why-does-altair-lead-to-such-extremely-large-notebooks

## Altair

In [ ]:
# Use "pip install altair"
#  mamba/conda wants to download: 1GB ?!
import altair as alt

In [ ]:
df = df.reset_index()
df.info()

In [ ]:
url = '301_CA_SNTL_slim.json'
#url = 'https://raw.githubusercontent.com/scottyhq/snotel/main/timeseries/301_CA_SNTL.json'
# https://github.com/pandas-dev/pandas/issues/25513
df.to_json(url, 
         orient='records', # Required for altair
         #date_unit='s',
         date_format='iso', #366K
         double_precision=5, # default 10:493K , 5:324K
        )

In [ ]:
!ls -lrh 301_CA_SNTL_slim.json

In [ ]:
!ls -lrh 301*

In [ ]:
#datetime unit = 's' {"datetime":1388534400,"TAVG":0.2,"TMIN":-6.7,"TMAX":10.6,"SNWD":0.0254,"WTEQ":0.0127,"PRCPSA":0.00254}
#datetime unit = 'ns'(default) {"datetime":1388534400000,"TAVG":0.2,"TMIN":-6.7,"TMAX":10.6,"SNWD":0.0254,"WTEQ":0.0127,"PRCPSA":0.00254},
#!head {url} 

In [ ]:
# NOTE this chart won't display in a jupyternotebook, need to save html
from pprint import pprint

chart = alt.Chart(url).mark_line().encode(
    x='datetime:T',
    y='SNWD:Q'
)
# NOTE: this does not appear in a jupyter notebook, but does render on a website!
chart

In [ ]:
pprint(chart.to_dict()) 

In [ ]:
chart.save('altair.html')

In [ ]:
ls -ltrh altair.html #only 1.4K! 

## Interactivitiy

In [ ]:
# Altair interactive
chart = alt.Chart(url).mark_line().encode(
    x='datetime:T',
    y='SNWD:Q',
    tooltip=['datetime:T','SNWD:Q'],
).interactive()
pprint(chart.to_dict()) # ValueError: datetime encoding field is specified without a type; the type cannot be automatically inferred because the data is not specified as a pandas.DataFrame.
#chart

In [ ]:
chart.save('altair-inteactive.html')

In [ ]:
# https://altair-viz.github.io/user_guide/data.html#including-index-data
chart = alt.Chart(df.reset_index(), title='301_CA_SNTL').mark_line().encode(
    x='datetime',
    y='WTEQ',
    #tooltip=['datetime','SNWD','WTEQ'],
)

#chart.interactive()
chart

In [ ]:
pprint(chart.to_dict()['encoding'])